In [1]:
from viewer import BallViewer
import jax 
import jax.numpy as np
from jax.numpy.linalg import pinv
import numpy as onp
import time 

In [2]:
viewer = BallViewer()
viewer.open()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x7f90925d3460> at path: <meshcat.path.Path object at 0x7f90f4da12a0>>

In [9]:


_m = 1. 
_r = 1.
_M = onp.diag([_m, _m, (2./3.) * _m * _r **2])
_M_inv = onp.linalg.inv(_M)
_g = 9.81
def f_analytical_ball(x):
    q, qdot = np.split(x, 2)
    b = np.array([0, -_g, 0])
    return np.concatenate([qdot, _M_inv @ b]) 

def phi(q):
    return q[1] - _r

def contact_jac(q):
    return np.array([
        [1.0, 0.0],
        [0.0, 1.0],
        [_r,  0.0]
    ])

def reset_map(q, qdot):
    J = contact_jac(q)
    print((np.eye(qdot.shape[0]) - 2*_M_inv@J.T@onp.diag([2, 1, 1])@pinv(J@_M_inv@J.T)@J).shape)
    return (np.eye(qdot.shape[0]) - 2*_M_inv@J.T@onp.diag([2, 1, 1])@pinv(J@_M_inv@J.T)@J)@qdot

q0    = np.array([0., 2.0, 0.])
qdot0 = np.array([0., 0., 1.0])
x     = np.concatenate([q0, qdot0])

for t in range(100):
    dxdt = f_analytical_ball(x)
    x  = x + 0.01*dxdt
    q, qdot = np.split(x, 2)
    if phi(q) <= 0:
        qdot = reset_map(q, qdot)
        x = np.concatenate([q,qdot])
    viewer.render(np.split(x,2)[0])
    time.sleep(0.01)

TypeError: dot_general requires contracting dimensions to have the same shape, got (3,) and (2,).